In [1]:
from math import atan, sqrt, degrees

In [2]:
map1 = '.#..#\
.....\
#####\
....#\
...##'

map2 = '......#.#.\
#..#.#....\
..#######.\
.#.#.###..\
.#..#.....\
..#....#.#\
#..#....#.\
.##.#..###\
##...#..#.\
.#....####'

map3 = '#.#...#.#.\
.###....#.\
.#....#...\
##.#.#.#.#\
....#.#.#.\
.##..###.#\
..#...##..\
..##....##\
......#...\
.####.###.'

map4 = '.#..#..###\
####.###.#\
....###.#.\
..###.##.#\
##.##.#.#.\
....###..#\
..#.#..#.#\
#..#.#.###\
.##...##.#\
.....#.#..'

map5 = '.#..##.###...#######\
##.############..##.\
.#.######.########.#\
.###.#######.####.#.\
#####.##.#.##.###.##\
..#####..#.#########\
####################\
#.####....###.#.#.##\
##.#################\
#####.##.###..####..\
..######..##.#######\
####.##.####...##..#\
.#####..#.######.###\
##...#.##########...\
#.##########.#######\
.####.#.###.###.#.##\
....##.##.###..#####\
.#.#.###########.###\
#.#.#.#####.####.###\
###.##.####.##.#..##'

map6 = '...###.#########.####\
.######.###.###.##...\
####.########.#####.#\
########.####.##.###.\
####..#.####.#.#.##..\
#.################.##\
..######.##.##.#####.\
#.####.#####.###.#.##\
#####.#########.#####\
#####.##..##..#.#####\
##.######....########\
.#######.#.#########.\
.#.##.#.#.#.##.###.##\
######...####.#.#.###\
###############.#.###\
#.#####.##..###.##.#.\
##..##..###.#.#######\
#..#..########.#.##..\
#.#.######.##.##...##\
.#.##.#####.#..#####.\
#.#.##########..#.##.'

In [3]:
class Asteroid:
    def __init__(self, x, y, map_list: list):
        self.coor = (x, y)
        self.scan = 0
        self.detected = {}
        self.vaped = {}
        self.map_list = map_list

    def search(self):
        for j in range(len(self.map_list)):

            for i in range(len(self.map_list)):
                if self.map_list[j][i] == '#' and (i, j) != self.coor and (i, j) not in self.vaped:
                    x_dir = i - self.coor[0]
                    y_dir = j - self.coor[1]

                    # Keys are directions, values are locations
                    # Account for asteroids in order of closeness to location
                    if lcd(x_dir, y_dir) not in self.detected:
                        self.detected[lcd(x_dir, y_dir)] = (i, j)
                        self.scan += 1
                        
                    else:
                        curr = self.detected[lcd(x_dir, y_dir)]
                        curr_dist = dist(curr[0], curr[1], self.coor[0], self.coor[1])
                        
                        if dist(i, j, self.coor[0], self.coor[1]) < curr_dist:
                            self.detected[lcd(x_dir, y_dir)] = (i, j)

In [4]:
def map_gen(map):
    length = int(sqrt(len(map)))
    map_list = [[map[i * length + j] for j in range(length)] for i in range(length)]

    return map_list

In [5]:
def dist(x0, y0, x1, y1):
    return sqrt((x1 - x0)**2 + (y1 - y0)**2)

In [6]:
# Get greatest common factor (gcf) between x and y
# Then divide x and y each by gcf to get lowest common denom
def gcf(a, b):
    if b == 0:
        return a
        
    return gcf(b, a % b)

def lcd(a, b):
    div = abs(gcf(a, b))  # Abs to avoid a negative / negative for positive

    return int(a/div), int(b/div) 

In [7]:
def best_most(map):
    map_list = map_gen(map)
    asteroids = []
    most_scanned = 0

    for y in range(len(map_list)):

        for x in range(len(map_list)):

            if map_list[y][x] == '#':
                asteroids.append(Asteroid(x, y, map_list))
                asteroids[-1].search()

                if asteroids[-1].scan > most_scanned:
                    most_scanned = asteroids[-1].scan
                    monitor = asteroids[-1]

    print('Best Location: {}'.format(monitor.coor))
    print('Asteroids Detected: {}'.format(most_scanned))
    print('')

    return monitor

In [8]:
def vaporize(map):
    monitor = best_most(map)
    destroyed = {}
    count = 0

    while monitor.detected != {}:
        angle = 359

        for item in monitor.detected:
            ast_id = monitor.detected[item]

            # Q1
            # Component division x/y... draw to see why
            if ast_id[0] >= monitor.coor[0] and ast_id[1] < monitor.coor[1]:
                curr_angle = degrees(atan(abs(item[0])/abs(item[1])))

            # Q2
            # Changes component division to y/x
            elif ast_id[0] > monitor.coor[0] and ast_id[1] >= monitor.coor[1]:
                curr_angle = degrees(atan(abs(item[1])/abs(item[0]))) + 90

            # Q3
            elif ast_id[0] <= monitor.coor[0] and ast_id[1] > monitor.coor[1]:
                curr_angle = degrees(atan(abs(item[0])/abs(item[1]))) + 180

            #Q4
            elif ast_id[0] < monitor.coor[0] and ast_id[1] <= monitor.coor[1]:
                curr_angle = degrees(atan(abs(item[1])/abs(item[0]))) + 270

            if curr_angle < angle:
                angle = curr_angle
                remove = item

        count += 1
        destroyed[count] = monitor.detected[remove]
        del monitor.detected[remove]
        
    return destroyed

In [9]:
def solve(number, map):
    solutions = vaporize(map)
    ans = solutions[number][0] * 100 + solutions[number][1]
    return ans

In [10]:
# Answer to Parts 1 & 2
print('Part 2 Answer: {}'.format(solve(200, map6)))

Best Location: (11, 13)
Asteroids Detected: 227

Part 2 Answer: 604
